Dependency Installation

In [ ]:
!py -m pip install matplotlib tensorflow scikit-learn opencv-python mediapipe

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow.compat.v1 as tf

Keypoints using MP Holistic


In [2]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #color conversion bgr/rgb
    image.flags.writeable = False                  # non writeable image
    results = model.process(image)                 # make prediction
    image.flags.writeable = True                   # writable image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #color conversion rgb/bgr
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [5]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
      ret, frame = cap.read()                #read feed
      image, results = mediapipe_detection(frame, holistic) 
      print(results)
      draw_styled_landmarks(image, results)         #draw landmarks
      cv2.imshow('OpenCV Feed', image)       #show to screen
      if cv2.waitKey(10) & 0xFF == ord('q'): #break gracefully
        break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

Extract Keypoint Values

In [6]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [7]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [8]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
    if results.face_landmarks 
    else np.zeros(1404)

IndentationError: unexpected indent (3987811915.py, line 2)

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh]) #([pose, face, lh, rh])

In [10]:
result_test = extract_keypoints(results)

Collection Folder Setup

In [11]:
# # Get the folder names in the DATA_PATH
# folders = os.listdir(actions)

# # Add the folder names to the actions array
# actions = np.concatenate((actions, folders))

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Hello', 'Goodbye', 'Good morning', 'Good night', 'I', 'You', 'He', 'She', 'It', 'We', 'They',
                    'Who', 'What', 'Where', 'When', 'Why', 'How', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                    'Eat', 'Drink', 'Go', 'Come', 'Run','Walk', 'See', 'Hear',
                    'Speak', 'Sleep','Man', 'Woman','I love you', 'Thank you'])




# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [12]:
# Create directories for each action
for action in actions:
    action_dir = os.path.join(DATA_PATH, action)
    if not os.path.exists(action_dir):
        os.makedirs(action_dir)

In [13]:
actions


array(['Hello', 'Goodbye', 'Good morning', 'Good night', 'I', 'You', 'He',
       'She', 'It', 'We', 'They', 'Who', 'What', 'Where', 'When', 'Why',
       'How', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Eat',
       'Drink', 'Go', 'Come', 'Run', 'Walk', 'See', 'Hear', 'Speak',
       'Sleep', 'Man', 'Woman', 'I love you', 'Thank you'], dtype='<U12')

In [14]:
30*30*3*1662

4487400

In [15]:
# hello
## 0
## 1
## 2
## ...
## 29
# thanks

# I love you

In [16]:

for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Collect Keypoint Values for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
        # Wait for a key press to proceed to the next action
        print("Press 'n' to proceed to the next action or 'q' to quit")
        while True:
            if cv2.waitKey(0) & 0xFF == ord('n'):
                break
            elif cv2.waitKey(0) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                exit()
                    
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
# # Set mediapipe model 
# mp_holistic = mp.solutions.holistic
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     # NEW LOOP
#     # Loop through actions
#     for action in actions:
#         # Get the list of frames for this action
#         frame_list = os.listdir(os.path.join(DATA_PATH, action))
        
#         # Loop through frames
#         for frame_name in frame_list:
#             # Read image from folder
#             frame = cv2.imread(os.path.join(DATA_PATH, action, frame_name))

#             # Make detections
#             image, results = mediapipe_detection(frame, holistic)

#             # Draw landmarks
#             draw_styled_landmarks(image, results)
                
#             # Apply wait logic
#             cv2.putText(image, 'Collecting frames for {}'.format(action), (15,12), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#             # Show to screen
#             cv2.imshow('OpenCV Feed', image)
                
#             # Export keypoints
#             keypoints = extract_keypoints(results)
#             npy_path = os.path.join('D:\Jupyter Notebooks\Convey\MP_Data', action, frame_name.split('.')[0])  # replace 'New_Directory' with your desired directory
#             os.makedirs(os.path.dirname(npy_path), exist_ok=True)  # create the directory if it doesn't exist
#             np.save(npy_path, keypoints)

#             # Break gracefully
#             if cv2.waitKey(10) & 0xFF == ord('q'):
#                 break
                    
#     cv2.destroyAllWindows()


In [ ]:
# # Specify the directory where the .npy files are
# data_dir = r'D:\Jupyter Notebooks\Convey\MP_Data'  # replace with your directory path

# # Loop through all actions in the directory
# for action in os.listdir(data_dir):
#     action_dir = os.path.join(data_dir, action)
    
#     # Check if it's a directory
#     if os.path.isdir(action_dir):
#         npy_files = [f for f in os.listdir(action_dir) if f.endswith('.npy')]
        
#         # Sort the files
#         npy_files.sort()
        
#         # Create a new subdirectory named '0'
#         new_subdir = os.path.join(action_dir, '0')
#         os.makedirs(new_subdir, exist_ok=True)
        
#         # Loop through all .npy files and rename them
#         for i, filename in enumerate(npy_files, start=0):
#             old_file = os.path.join(action_dir, filename)
#             new_file = os.path.join(new_subdir, f"{i}.npy")
            
#             # Move and rename the file
#             os.rename(old_file, new_file)


In [ ]:
cap.release()
cv2.destroyAllWindows()

Preprocess Data and Create Labels and Features

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [18]:
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
label_map

{'Hello': 0,
 'Goodbye': 1,
 'Good morning': 2,
 'Good night': 3,
 'I': 4,
 'You': 5,
 'He': 6,
 'She': 7,
 'It': 8,
 'We': 9,
 'They': 10,
 'Who': 11,
 'What': 12,
 'Where': 13,
 'When': 14,
 'Why': 15,
 'How': 16,
 '1': 17,
 '2': 18,
 '3': 19,
 '4': 20,
 '5': 21,
 '6': 22,
 '7': 23,
 '8': 24,
 '9': 25,
 '10': 26,
 'Eat': 27,
 'Drink': 28,
 'Go': 29,
 'Come': 30,
 'Run': 31,
 'Walk': 32,
 'See': 33,
 'Hear': 34,
 'Speak': 35,
 'Sleep': 36,
 'Man': 37,
 'Woman': 38,
 'I love you': 39,
 'Thank you': 40}

In [20]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])



In [21]:
np.array(sequences)

array([[[ 0.74513829,  0.35297501, -0.88064605, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55038613,  0.34408551, -0.77222216, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.53582907,  0.33724731, -0.77980816, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.52738965,  0.34510583, -1.09216142, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.52749062,  0.34420687, -1.08049297, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.52763057,  0.34357068, -1.0707705 , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.52759242,  0.34168294, -1.07784915, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55771673,  0.33277193, -0.70949394, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.57182765,  0.32888645, -0.65019679, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.45213476,  0.32967472, -0.96152037, ...,  

In [22]:
np.array(labels).shape

(1230,)

In [23]:
X = np.array(sequences)

In [24]:
X.shape

(1230, 30, 258)

In [25]:
y = to_categorical(labels).astype(int)

In [26]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [28]:
y_test.shape

(62, 41)

Build and Train LSTM Neural Network

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [30]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [31]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258))) #1662
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [32]:
res = [.7, 0.2, 0.1]

In [33]:
actions[np.argmax(res)]

'Hello'

In [34]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [63]:
model.fit(X_train, y_train, epochs=10, callbacks=[tb_callback])

Epoch 1/10
37/37 [==============================] - 2s 49ms/step - loss: 0.2394 - categorical_accuracy: 0.9195
Epoch 2/10
37/37 [==============================] - 2s 47ms/step - loss: 0.4076 - categorical_accuracy: 0.8630
Epoch 3/10
37/37 [==============================] - 2s 48ms/step - loss: 0.3008 - categorical_accuracy: 0.8938
Epoch 4/10
37/37 [==============================] - 2s 47ms/step - loss: 0.2277 - categorical_accuracy: 0.9264
Epoch 5/10
37/37 [==============================] - 2s 47ms/step - loss: 0.1414 - categorical_accuracy: 0.9580
Epoch 6/10
37/37 [==============================] - 2s 49ms/step - loss: 0.2439 - categorical_accuracy: 0.9170
Epoch 7/10
 8/37 [=====>........................] - ETA: 1s - loss: 0.2309 - categorical_accuracy: 0.9180

KeyboardInterrupt: 

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 41)                1353      
                                                                 
Total params: 238505 (931.66 KB)
Trainable params: 23850

Predictions

In [55]:
res = model.predict(X_test)

2/2 [==============================] - 0s 37ms/step


In [56]:
actions[np.argmax(res[4])]

'She'

In [57]:
actions[np.argmax(y_test[4])]

'She'

Save Weights

In [39]:
model.save('40_actions.h5')

C:\Users\avani\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
del model

In [29]:
model.load_weights('40_actions.h5')

Evaluation using Confusion Matrix and Accuracy

In [58]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [64]:
yhat = model.predict(X_test)

2/2 [==============================] - 0s 31ms/step


In [65]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [66]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[60,  0],
        [ 0,  2]],

       [[61,  0],
        [ 0,  1]],

       [[61,  0],
        [ 0,  1]],

       [[60,  0],
        [ 1,  1]],

       [[60,  0],
        [ 0,  2]],

       [[60,  1],
        [ 0,  1]],

       [[61,  1],
        [ 0,  0]],

       [[59,  1],
        [ 0,  2]],

       [[59,  1],
        [ 1,  1]],

       [[60,  0],
        [ 1,  1]],

       [[61,  0],
        [ 0,  1]],

       [[61,  0],
        [ 0,  1]],

       [[60,  0],
        [ 0,  2]],

       [[61,  1],
        [ 0,  0]],

       [[57,  0],
        [ 3,  2]],

       [[61,  0],
        [ 0,  1]],

       [[58,  4],
        [ 0,  0]],

       [[57,  1],
        [ 4,  0]],

       [[60,  0],
        [ 1,  1]],

       [[60,  1],
        [ 1,  0]],

       [[60,  0],
        [ 0,  2]],

       [[59,  0],
        [ 1,  2]],

       [[60,  0],
        [ 0,  2]],

       [[60,  0],
        [ 1,  1]],

       [[59,  1],
        [ 0,  2]],

       [[60,  2],
        [ 0,  0]],

       [[61,

In [67]:
accuracy_score(ytrue, yhat)

0.7258064516129032

Test in Real Time

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
sequence

In [ ]:
len(sequence)

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 3: 
                sentence = sentence[-3:]

            # # Viz probabilities
            # image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))

RERUN ONLY BELOW CELLS FOR MODEL LOAD

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow.compat.v1 as tf

mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #color conversion bgr/rgb
    image.flags.writeable = False                  # non writeable image
    results = model.process(image)                 # make prediction
    image.flags.writeable = True                   # writable image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #color conversion rgb/bgr
    return image, results

def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )



In [2]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
      ret, frame = cap.read()                #read feed
      image, results = mediapipe_detection(frame, holistic) 
      print(results)
      draw_styled_landmarks(image, results)         #draw landmarks
      cv2.imshow('OpenCV Feed', image)       #show to screen
      if cv2.waitKey(10) & 0xFF == ord('q'): #break gracefully
        break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh]) #([pose, face, lh, rh])

result_test = extract_keypoints(results)

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Hello', 'Goodbye', 'Good morning', 'Good night', 'I', 'You', 'He', 'She', 'It', 'We', 'They',
                    'Who', 'What', 'Where', 'When', 'Why', 'How', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                    'Eat', 'Drink', 'Go', 'Come', 'Run','Walk', 'See', 'Hear',
                    'Speak', 'Sleep','Man', 'Woman','I love you', 'Thank you'])

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)

X.shape

y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258))) #1662
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


res = [.7, 0.2, 0.1]

actions[np.argmax(res)]

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.load_weights('40_actions.h5')

res = model.predict(X_test)

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.95

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()                #read feed
        image, results = mediapipe_detection(frame, holistic) 

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 3: 
                sentence = sentence[-3:]

            # # Viz probabilities
            # image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()





2/2 [==============================] - 0s 24ms/step
